In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from scipy.io import loadmat
from PIL import Image
import cv2
from tqdm.notebook import tqdm

In [2]:
matfile = loadmat("../original_data/imdb-animalParts-eye.mat")

In [3]:
matfile.keys()

dict_keys(['__header__', '__version__', '__globals__', 'images', 'bbx', 'kp', 'classes', 'imageDir'])

In [4]:
image_names = matfile["images"]["name"][0][0][0]
image_id = matfile["images"]["id"][0][0][0]
bbx_id = matfile["bbx"]["id"][0][0][0]
bbx_imageId = matfile["bbx"]["imageId"][0][0][0]
kp_boxId = matfile["kp"]["boxId"][0][0][0]
kp_pos = matfile["kp"]["pos"][0][0].T

In [5]:
len(image_names)

14711

In [6]:
rh, rw = 256, 256

In [7]:
im_count = 0
eye_pos_shifts = []
kp_img_id = []
for img_idx in tqdm(range(len(image_names))):
    im_name = image_names[img_idx][0]
    im_dir = im_name.split("/")[0]
    if im_dir == "val":
        img_path = f"/data/dataset/imagenet/ILSVRC2012_{image_names[img_idx][0]}"
        im = cv2.imread(img_path)
        """
        if im.mode == "L":
            im = im.convert('RGB')
        im = np.array(im)
        """
        im_id = image_id[img_idx]
        kp_indices = bbx_id[im_id == bbx_imageId][0] == kp_boxId
        num_kp = np.sum(kp_indices)
        if num_kp > 0:
            eye_pos = kp_pos[kp_indices]
            mean_eye_pos = np.mean(eye_pos, axis=0)
            h, w, _ = im.shape
            eye_pos_shift = np.copy(eye_pos)
            if h == w:
                min_hw = h
                im_crop_resize = cv2.resize(im, (rw, rh), interpolation=cv2.INTER_AREA)
            else:
                min_hw = min(h, w)
                if h == min_hw:
                    x = np.arange(0, w - min_hw)
                    xm = x + w/2
                    min_idx = np.argmin(np.abs(mean_eye_pos[0] - xm))
                    cx = x[min_idx]
                    im_crop = im[:, cx:cx+min_hw]
                    eye_pos_shift[:, 0] -= cx
                else:
                    y = np.arange(0, h - min_hw)
                    ym = y + h/2
                    min_idx = np.argmin(np.abs(mean_eye_pos[1] - ym))
                    cy = y[min_idx]
                    im_crop = im[cy:cy+min_hw]
                    eye_pos_shift[:, 1] -= cy
                im_crop_resize = cv2.resize(im_crop, (rw, rh), interpolation=cv2.INTER_AREA)
            eye_pos_shift[:, 0] *= rw / min_hw
            eye_pos_shift[:, 1] *= rh / min_hw
            eye_pos_shifts.append(eye_pos_shift)
            kp_img_id += [im_count] * num_kp
    
            cv2.imwrite(f"../preprocessed_data/resized_images_val/{im_count:05d}.png", im_crop_resize)
            im_count += 1
eye_pos_shifts = np.vstack(eye_pos_shifts)
kp_img_id = np.hstack(kp_img_id)

  0%|          | 0/14711 [00:00<?, ?it/s]

In [9]:
im_count

1067

In [8]:
np.savez_compressed("../preprocessed_data/shifted_eye_pos_val.npz", 
                    eye_pos=eye_pos_shifts, img_id=kp_img_id)

In [10]:
kp_img_id

array([   0,    1,    1, ..., 1064, 1065, 1066])